In [1]:
import random
import numpy as np
import pandas as pd
import duckdb

Autre exemple: assurance santé. <br />
On pourrait vouloir grouper:
- par type de soin
- par type de contrat
- par groupe d'age du bénéficiaire
- et par toutes les combinaisons possibles des groupes ci-dessus:
- par type de soin et groupe d'age
- par type de contrat et groupe d'age
- par type de soin et type de contrat
- par type de soin, contrat, et groupe d'âge

# Data

In [2]:
random.seed(42)
num_samples = 1000

contrats = ["senior", "jeunes", "expat", "famille", "salarié"]
sexe = ["homme", "femme"]
type_acte = {"pharmacie": 15,
"consultation_generaliste": 25,
"hospitalisation": 2800,
"biologie": 150,
"radio": 1300,
"maternite": 1700}
groupe_age = ["18-25", "25-45", "45-65", "65+"]
annee = [2017, 2018, 2019]


# Initialize empty lists to store the data
contrats_data = []
sexe_data = []
type_acte_data = []
groupe_age_data = []
annee_data = []
cost_data = []

# Generate random data for each category
for _ in range(num_samples):
    contrats_data.append(random.choice(contrats))
    sexe_data.append(random.choice(sexe))
    if sexe_data == "femme":
        type_acte_choice = random.choice(list(type_acte.keys()))
    else:
        type_acte_options = list(type_acte.keys())
        type_acte_options.remove("maternite")
        type_acte_choice = random.choice(type_acte_options)
        
    type_acte_data.append(type_acte_choice)
    cost_mean = type_acte[type_acte_choice]
    cost_data.append(np.random.normal(cost_mean, cost_mean // 3.5))  # Assuming a standard deviation of 50 for costs
    groupe_age_data.append(random.choice(groupe_age))
    annee_data.append(random.choice(annee))


In [3]:
# Create a DataFrame to store the dataset
df = pd.DataFrame({
    'type_contrat': contrats_data,
    'sexe': sexe_data,
    'type_acte': type_acte_data,
    'groupe_age': groupe_age_data,
    'annee': annee_data,
    'montant_rembourse': cost_data
})
df

,type_contrat,sexe,type_acte,groupe_age,annee,montant_rembourse
0,senior,homme,hospitalisation,25-45,2017,1048.706450
1,jeunes,homme,radio,18-25,2019,1760.258201
2,famille,homme,pharmacie,18-25,2017,13.354040
3,jeunes,homme,radio,25-45,2019,1977.437187
4,salarié,femme,consultation_generaliste,65+,2019,23.485193
...,...,...,...,...,...,...
995,famille,homme,biologie,45-65,2018,139.953504
996,jeunes,homme,hospitalisation,65+,2019,1038.384114
997,salarié,femme,pharmacie,18-25,2017,16.734192
998,expat,homme,hospitalisation,45-65,2019,2148.091239


# Exercices: Rollup

Voici le montant global remboursé par la caisse d'assurance sur la période 2017-2019

In [5]:
query = """
SELECT SUM(montant_rembourse) 
FROM df
"""
duckdb.sql(query)

┌────────────────────────┐
│ sum(montant_rembourse) │
│         double         │
├────────────────────────┤
│       865056.458683528 │
└────────────────────────┘

In [4]:
query = """
SELECT type_contrat, SUM(montant_rembourse) 
FROM df
GROUP BY type_contrat
"""
duckdb.sql(query)

┌──────────────┬────────────────────────┐
│ type_contrat │ sum(montant_rembourse) │
│   varchar    │         double         │
├──────────────┼────────────────────────┤
│ famille      │     180664.34890717122 │
│ expat        │     151054.63620484914 │
│ senior       │     179128.85501645587 │
│ salarié      │      179906.5258142123 │
│ jeunes       │     157321.51761110974 │
└──────────────┴────────────────────────┘

Votre PO vous demande le montant total remboursé par type de contrat

In [ ]:
# %load solutions/7groupby_type_contrat.py
query = """
SELECT type_contrat, SUM(montant_rembourse) 
FROM df
GROUP BY type_contrat
"""
duckdb.sql(query)


Votre Manager vous transmet une autre demande du PO: <br />
il vous demande le montant total remboursé par type de contrat
ET par type d'acte

In [9]:
query = """
SELECT type_contrat, type_acte, SUM(montant_rembourse) 
FROM df
GROUP BY ROLLUP(type_contrat, type_acte)
HAVING type_contrat IS NOT NULL AND type_acte IS NOT NULL
ORDER BY type_contrat, type_acte
"""
duckdb.sql(query)

┌──────────────┬──────────────────────────┬────────────────────────┐
│ type_contrat │        type_acte         │ sum(montant_rembourse) │
│   varchar    │         varchar          │         double         │
├──────────────┼──────────────────────────┼────────────────────────┤
│ expat        │ biologie                 │      6876.624727429615 │
│ expat        │ consultation_generaliste │     1129.7734511271708 │
│ expat        │ hospitalisation          │      89685.08787490273 │
│ expat        │ pharmacie                │      564.2653322824361 │
│ expat        │ radio                    │     52798.884819107065 │
│ famille      │ biologie                 │      7633.276635832385 │
│ famille      │ consultation_generaliste │      778.3530676762116 │
│ famille      │ hospitalisation          │     110181.59779489809 │
│ famille      │ pharmacie                │     446.18662653958546 │
│ famille      │ radio                    │      61624.93478222487 │
│    ·         │   ·              

In [ ]:
# %load solutions/8groupby_typecontrat_typeacte.py
query = """
SELECT type_contrat, type_acte, SUM(montant_rembourse) 
FROM df
GROUP BY type_contrat, type_acte
ORDER BY type_contrat
"""
duckdb.sql(query)


Le PO revient en râlant: le manager n'avait rien compris ! <br />
Il veut le montant total remboursé par type de contrat <br />
ET le montant total remboursé par type d'acte

Hint: commencez simple, faites le avec un Union

In [15]:
# %load solutions/9groupby_unions.py
query = """
SELECT type_contrat AS typologie,
SUM(montant_rembourse) 
FROM df
GROUP BY type_contrat

UNION

SELECT type_acte AS typologie, 
SUM(montant_rembourse) 
FROM df
GROUP BY type_acte
"""
duckdb.sql(query)


┌──────────────────────────┬────────────────────────┐
│        typologie         │ sum(montant_rembourse) │
│         varchar          │         double         │
├──────────────────────────┼────────────────────────┤
│ senior                   │     179128.85501645587 │
│ expat                    │     151054.63620484914 │
│ radio                    │     271236.95385288267 │
│ salarié                  │      179906.5258142123 │
│ jeunes                   │     157321.51761110974 │
│ consultation_generaliste │      5140.012224142445 │
│ biologie                 │       32605.3268670868 │
│ hospitalisation          │      536468.9339651847 │
│ pharmacie                │     2624.6566445016624 │
│ famille                  │     180664.34890717122 │
├──────────────────────────┴────────────────────────┤
│ 10 rows                                 2 columns │
└───────────────────────────────────────────────────┘

Votre tech lead vient vous voir:

<blockquote> Ton code fait le taff, mais j'ai récemment lu un article sur les GROUPING SETS, je pense que ça permettrait de simplifier le code sur ton problème, tu peux implémenter ça ? Merci ! </blockquote>

In [17]:
# %load solutions/10groupby_groupingsets.py
query = """
SELECT type_contrat, type_acte, SUM(montant_rembourse) 
FROM df
GROUP BY GROUPING SETS
(type_contrat, type_acte)
-- Notez la difference avec ((type_contrat, type_acte))
ORDER BY type_contrat, type_acte
"""
duckdb.sql(query)


┌──────────────┬──────────────────────────┬────────────────────────┐
│ type_contrat │        type_acte         │ sum(montant_rembourse) │
│   varchar    │         varchar          │         double         │
├──────────────┼──────────────────────────┼────────────────────────┤
│ expat        │ NULL                     │     151054.63620484914 │
│ famille      │ NULL                     │     180664.34890717122 │
│ jeunes       │ NULL                     │     157321.51761110974 │
│ salarié      │ NULL                     │      179906.5258142123 │
│ senior       │ NULL                     │     179128.85501645587 │
│ NULL         │ biologie                 │       32605.3268670868 │
│ NULL         │ consultation_generaliste │      5140.012224142445 │
│ NULL         │ hospitalisation          │      536468.9339651847 │
│ NULL         │ pharmacie                │     2624.6566445016624 │
│ NULL         │ radio                    │     271236.95385288267 │
├──────────────┴──────────────────

Votre manager revient à la charge:
<blockquote> Perso, je trouve que c'était très bien d'avoir aussi le montant total remboursé par type de contrat
ET par type d'acte. J'ai discuté avec le PO et il est OK. Garde ce que t'as fait, mais remets aussi les subdivisions </blockquote>

Vous décidez d'utiliser ROLLUP pour obtenir tout ça facilement:

In [19]:
# %load solutions/11rollup.py
query = """
SELECT type_contrat, type_acte, SUM(montant_rembourse) 
FROM df
GROUP BY ROLLUP
(type_contrat, type_acte)
ORDER BY type_contrat, type_acte
"""
duckdb.sql(query).df()


,type_contrat,type_acte,sum(montant_rembourse)
0,expat,biologie,6876.624727
1,expat,consultation_generaliste,1129.773451
2,expat,hospitalisation,89685.087875
3,expat,pharmacie,564.265332
4,expat,radio,52798.884819
5,expat,None,151054.636205
6,famille,biologie,7633.276636
7,famille,consultation_generaliste,778.353068
8,famille,hospitalisation,110181.597795
9,famille,pharmacie,446.186627


# Exercices: CUBE

Votre tech lead vérifie votre code et vous dit:

<blockquote> Tu t'es planté. ROLLUP ça enlève progressivement le niveau de regroupement le plus à droite de ta liste. Résultat: on n'a pas les sommes par type_acte uniquement (il manque 5 lignes) 
Il faut trouver une autre solution ! </blockquote>


Il nous faut donc:
- la somme par type d'acte (5 lignes),
- la somme par type de contrat (5 lignes),
- ET la somme par type d'acte + type de contrat (25 lignes),

Et effectivement, avec ROLLUP il nous manquait la somme par type d'acte uniquement (5 lignes)

Vous décidez d'utiliser un CUBE Pour avoir toutes les options:

In [21]:
# %load solutions/12cube.py
query = """
SELECT type_contrat, type_acte, SUM(montant_rembourse) 
FROM df
GROUP BY CUBE
(type_contrat, type_acte)
ORDER BY type_contrat, type_acte
"""
duckdb.sql(query).df()


,type_contrat,type_acte,sum(montant_rembourse)
0,expat,biologie,6876.624727
1,expat,consultation_generaliste,1129.773451
2,expat,hospitalisation,89685.087875
3,expat,pharmacie,564.265332
4,expat,radio,52798.884819
5,expat,None,151054.636205
6,famille,biologie,7633.276636
7,famille,consultation_generaliste,778.353068
8,famille,hospitalisation,110181.597795
9,famille,pharmacie,446.186627


# Grand' Final

En lisant le rapport fourni par votre P.O. sur la base de vos chiffres <br />
Votre N+2 a eu plein d'idées.

<img src="images/asterix_plein_idees.gif" />
<p style="text-align:center"><i>Le N+2</i></p>

<blockquote> Je veux la somme des montants remboursés par:
 type_contrat / type_acte / groupe_age / sexe / annee
</blockquote>

et je veux avoir les chiffres globaux pour chacune de ces sous-catégories:
- type_contrat / type_acte / groupe_age / sexe /
- type_contrat / type_acte / groupe_age /
- type_contrat / type_acte / 
- type_contrat / 

Maintenant que vous connaissez ROLLUP, ça devrait être un jeu d'enfant:

In [23]:
# %load solutions/13rollup_finale.py
query = """
SELECT type_contrat, type_acte, groupe_age, sexe, annee,
SUM(montant_rembourse) 
FROM df
GROUP BY ROLLUP
( type_contrat, type_acte, groupe_age, sexe, annee )
ORDER BY type_contrat, type_acte, groupe_age, sexe, annee 
"""
duckdb.sql(query).df().head(15)


,type_contrat,type_acte,groupe_age,sexe,annee,sum(montant_rembourse)
0,expat,biologie,18-25,femme,2017,144.710704
1,expat,biologie,18-25,femme,2018,131.251928
2,expat,biologie,18-25,femme,2019,98.599122
3,expat,biologie,18-25,femme,<NA>,374.561753
4,expat,biologie,18-25,homme,2017,141.717435
5,expat,biologie,18-25,homme,2018,146.866557
6,expat,biologie,18-25,homme,2019,654.836824
7,expat,biologie,18-25,homme,<NA>,943.420815
8,expat,biologie,18-25,None,<NA>,1317.982569
9,expat,biologie,25-45,femme,2017,240.482241
